In [1]:
import os
from tensorflow.python.platform import gfile
import scipy.io.wavfile as wav
from python_speech_features import *
import numpy as np
from subprocess import call
from struct import unpack, pack

DATA_DIR = os.path.join("/n/sd7/trung/csp", "data", "vivos")

/home/trungdv/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
mode = "train"

wset = set()
with open(os.path.join(DATA_DIR, mode, "prompts.txt"), encoding="utf-8") as f:
    for s in f.read().split('\n'):
        words = s.split(' ')[1:]
        for word in words: wset.add(word.lower())

print(len(wset))
with open(os.path.join(DATA_DIR, "words.txt"), "w", encoding="utf-8") as f:
    f.write("<unk> 0\n<sos> 1\n<eos> 2\n")
    f.write("\n".join(["%s %d" % (word, i + 2) for i, word in
        enumerate(list(wset)) if word != ""]))

4862


In [3]:
# get mean
mean = np.array([0] * 120)
var = np.array([0] * 120)
count = 0
with open(os.path.join(DATA_DIR, "train", "prompts.txt"), encoding="utf-8") as f:
    lines = f.read().split("\n")
    for i, s in enumerate(lines):
        filename = s.split(' ')[0]
        if filename == "": continue
        d[filename] = ' '.join(s.split(' ')[1:])
        wav_filename = os.path.join(DATA_DIR, "train", "waves", filename.split('_')[0], filename + ".wav")
        npy_filename = os.path.join(DATA_DIR, "train", "npy", filename.split('_')[0], filename + ".npy")
        (rate, sig) = wav.read(wav_filename)

        filename = os.path.join(DATA_DIR, "train", "feature", filename + ".htk")
        call([
            "/n/sd7/trung/bin/htk/HTKTools/HCopy",
            wav_filename,
            filename,
            "-C", "/n/sd7/trung/config.lmfb.40ch"
        ])
        fh = open(filename, "rb")
        spam = fh.read(12)
        nSamples, sampPeriod, sampSize, parmKind = unpack(">IIHH", spam)
        veclen = int(sampSize / 4)
        fh.seek(12, 0)
        dat = np.fromfile(fh, dtype=np.float32)
        dat = dat.reshape(len(dat) // veclen, veclen)
        dat = dat.byteswap()
        fh.close()
        
        for k in range(len(dat)):
            updated_mean = (mean * count + dat[k]) / (count + 1)
            var = (count * var + (dat[k] - mean) * (dat[k] - updated_mean)) / (count + 1)
            mean = updated_mean
            count += 1

        outputs.append(
            (npy_filename,
            ' '.join(['2'] + [words[w.lower()] if w.lower() in words else \
                '0' for w in s.split(' ')[1:]] + ['1']))
        )
            
        if i % 100 == 1: print("%d / %d" % (i, len(lines)))
            
np.save(os.path.join(DATA_DIR, "mean.npy"), mean)
np.save(os.path.join(DATA_DIR, "var.npy"), var)
print(mean)

NameError: name 'd' is not defined

In [6]:
mean = np.load(os.path.join(DATA_DIR, "mean.npy"))
var = np.load(os.path.join(DATA_DIR, "var.npy"))

def get_features_npy(infile, outfile):
    fh = open(infile, "rb")
    spam = fh.read(12)
    nSamples, sampPeriod, sampSize, parmKind = unpack(">IIHH", spam)
    veclen = int(sampSize / 4)
    fh.seek(12, 0)
    dat = np.fromfile(fh, dtype=np.float32)
    dat = dat.reshape(len(dat) // veclen, veclen)
    dat = dat.byteswap()
    fh.close()
    
    dat = (dat - mean) / np.sqrt(var)
    # print(len(dat), dat)
    np.save(outfile, dat)

In [7]:
mode = "test"

d = {}
words = open(os.path.join(DATA_DIR, "words.txt")).read().split('\n')
words = { word.split(' ')[0]: word.split(' ')[1] for word in words if word != ""}
outputs = []
with open(os.path.join(DATA_DIR, mode, "prompts.txt"), encoding="utf-8") as f:
    lines = f.read().split("\n")
    for i, s in enumerate(lines):
        filename = s.split(' ')[0]
        if filename == "": continue
        d[filename] = ' '.join(s.split(' ')[1:])
        wav_filename = os.path.join(DATA_DIR, mode, "waves", filename.split('_')[0], filename + ".wav")
        npy_filename = os.path.join(DATA_DIR, mode, "npy", filename + ".npy")
        (rate, sig) = wav.read(wav_filename)

        filename = os.path.join(DATA_DIR, mode, "feature", filename + ".htk")
        #call([
        #    "/n/sd7/trung/bin/htk/HTKTools/HCopy",
        #    "-C", "/n/sd7/trung/config.lmfb.40ch", "-T", "1",
        #    wav_filename, filename
        #])
        get_features_npy(filename, npy_filename)

        outputs.append(
            (npy_filename,
            ' '.join(['2'] + [words[w.lower()] if w.lower() in words else \
                '0' for w in s.split(' ')[1:]] + ['1']))
        )
            
        if i % 100 == 1: print("%d / %d" % (i, len(lines)))

outputs.sort(key=lambda x: len(x[1]))
with open(os.path.join(DATA_DIR, mode, "data.txt"), "w", encoding="utf-8") as f:
        f.write('\n'.join(["%s %s" % (fn, tg) for fn, tg in outputs]))

1 / 761
101 / 761
201 / 761
301 / 761
401 / 761
501 / 761
601 / 761
701 / 761
